<a href="https://colab.research.google.com/github/Lursen/Ensemble-methods/blob/main/Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загрузка данных, инициализация весов каждого примера выборки

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#bool_in=0
#real_in=51
#bool_out=2
#real_out=0
#training_examples=345
#validation_examples=173
#test_examples=172

data = np.loadtxt('card2.dt')
X = data[:, 0:51]
Y = data[:, 52]

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.9, random_state=42)
# Weights of samples
X_weights = np.zeros(np.shape(X_train)[0])
X_weights = X_weights + 1/np.shape(X_train)[0]

Класс сети прямого распространения

In [ ]:
import torch 
import matplotlib.pyplot as plt
import numpy as np

class Feedforward(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()
            self.tanh = torch.nn.Tanh()

        def forward(self, x):
            hidden = self.fc1(x)
            sigmoid = self.sigmoid(hidden)
            output = self.fc2(sigmoid)
            output = self.sigmoid(output)
            return output

Функции обучения комитета

In [ ]:
import random
def train_commitee(m, x,y,w, x_test,y_test):
  NN = [[] for i in range(m)]
  NN_accuracy = [[] for i in range(m)]
  NN_weight_err = [[] for i in range(m)]

  samples_w = np.asarray(w)
  samples_x = np.asarray(x)
  samples_y = np.asarray(y)
  t = 51

  for i in range(0,m):
    NN[i] = Feedforward(t, 50)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(NN[i].parameters(), lr = 0.1)
    NN_errors = []
    NN_weights = []
    NN_werr = 0
    # training
    NN[i].train()
    NN[i].double()
    epoch = 1000

    for epoch in range(epoch):
      rnd = np.random.randint(0,np.shape(samples_x)[0])
      xs = torch.from_numpy(samples_x[rnd])
      ys = torch.tensor(samples_y[rnd])
      optimizer.zero_grad()
      # Forward pass
      y_pred = NN[i](xs)
      # Compute Loss
      loss = criterion(y_pred.squeeze(), ys)
      # Get binary answer
      answer = 0
      if (y_pred.item() >= 0.8):
        answer = 1
      # weighted loss
      r = 1
      if (answer == ys.item()):
        r = 0
      weighted_loss = samples_w[rnd]*r

      NN_errors.append(weighted_loss)
      NN_weights.append(samples_w[rnd])
      NN_accuracy[i].append(1-np.abs(y_pred.item() - ys.item()))
      # Backward pass
      loss.backward()
      optimizer.step()
      
    em = (np.sum(np.asarray(NN_errors))/np.sum(np.asarray(NN_weights)))
    NN_werr = np.log((1.0-em)/em) 
    NN_weight_err[i] = NN_werr
    #print(NN_weight_err[i])
    # compute sample error and new weights
    for s in range(samples_y.shape[0]):
      xs = torch.from_numpy(x[s])
      y_pred = NN[i](xs)
      answer = 0
      if (y_pred.item() >= 0.8):
        answer = 1
      samples_err = 0
      if (answer==y[s]):
        samples_err = 1
      samples_w[s] = samples_w[s]*np.exp(NN_werr*samples_err)

    # create new training set
    rn = list(range(0,samples_y.shape[0]))
    samples_idx = random.choices(rn, weights=samples_w, k = samples_y.shape[0])
    for s in range(samples_y.shape[0]):
      samples_x[s] = x[samples_idx[s]]
      samples_y[s] = y[samples_idx[s]]

  accuracy = [[] for i in range(m)]
  for i in range(0,m):
    for t in range(0,100):
      rnd = np.random.randint(0,np.shape(x_test)[0])
      xs = torch.from_numpy(x_test[rnd])
      y_pred = NN[i](xs)
      acc =  1-np.abs(y_pred.item() - y_test[rnd])
      NN_accuracy[i].append(acc)
  
    accuracy[i] = np.mean(np.asarray(NN_accuracy[i]))

  return NN, accuracy, NN_weight_err

def get_commitee_accuracy(NN, cl_err, x_test, y_test):
  accuracy = []

  for t in range(100):
    rnd = np.random.randint(0,np.shape(x_test)[0])
    xs = torch.from_numpy(x_test[rnd])
    answer = 0
    for i in range(0,np.shape(NN)[0]):
      y_pred = NN[i](xs)
      answer = answer + y_pred#(np.abs(cl_err[i])*y_pred.item())

    if (answer/np.shape(NN)[0] >= 0.6):
      answer = 1
    else:
      answer = 0
    accuracy.append(1-np.abs(answer - y_test[rnd]))

  acc_mean = np.mean(np.asarray(accuracy))
  std = np.std(np.asarray(accuracy))
  return acc_mean, std

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#bool_in=0
#real_in=51
#bool_out=2
#real_out=0
#training_examples=345
#validation_examples=173
#test_examples=172

data = np.loadtxt('card2.dt')
X = data[:, 0:51]
Y = data[:, 52]

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.9, random_state=42)
# Weights of samples
X_weights = np.zeros(np.shape(X_train)[0])
X_weights = X_weights + 1/np.shape(X_train)[0]

7 Классификаторов

In [ ]:
m = 7
NN, accuracy, cl_err = train_commitee(m, X_train, y_train, X_weights, X_test, y_test)
com_acc, std = get_commitee_accuracy(NN, cl_err, X_test, y_test)
print('acc:',accuracy)
print('com_acc:',com_acc)
print('std:',std)


acc: [0.5831759310449434, 0.5853782560044642, 0.6164123029231402, 0.6410719598941661, 0.6544995382675183, 0.6584421225812573, 0.6543249882410295]
com_acc: 0.83
std: 0.375632799419859
